In [1]:
## FOR GENERAL UNIX FUNCTIONS SEE trace-unix-general.ipynb 
## STARTS FROM LINE 1687 (tomohara-aliases.bash)

## Bracketed Paste is disabled to prevent characters after output
## Example: 
# $ echo 'Hi'
# | Hi?2004l

bind 'set enable-bracketed-paste off'

In [2]:
## TEST:  Make sure simple prompt used (e.g., no escapes that might contaminate output)
## PS1="$ "
## TODO: PS1="> "

## NOTE: The Jupyter bash kernel requires that PS1 not be modified as they customize it. 
echo $PS1

[PEXP\[\]ECT_PROMPT>


In [3]:
# Delete all aliases and function
# TODO: Instead start from pristine environment
unalias -a
alias | wc -l
for f in $(typeset -f | egrep '^\w+'); do unset -f $f; done
typeset -f | egrep '^\w+' | wc -l

0
0


In [4]:
# Setting a temp directory for tests
TMP=/tmp/test-unix-alias

## NOTE: Source it directly from the ./tests directory.
BIN_DIR=$PWD/..

alias | wc -l

0


In [5]:
## NOTE: For reproducability, the directory name needs to be fixed
## In place of $$, use a psuedo random number (e,g., 2899)
## *** All output from one run to the next needs to be the same ***

## temp_dir=$TMP/test-$$
temp_dir=$TMP/test-2899

mkdir -p "$temp_dir"
# TODO: /bin/rm -rvf "$temp_dir"
cd "$temp_dir"
pwd

#ALIAS FOR PRINTING SEPERATION LINES (FOR JUPYTER)
alias linebr="printf '%*s\n' "${COLUMNS:-$(tput cols)}" '' | tr ' ' -"

/tmp/test-unix-alias/test-2899


In [6]:
# Count aliases proper
alias | wc -l
# Count functions
typeset -f | egrep '^\w+' | wc -l

1
0


In [11]:
function group-members () { ypcat group | $GREP -i "$1"; }

In [12]:
## ERROR I - group-members

# $ group-members
# | bash: ypcat: command not found
# | bash: -i: command not found

In [21]:
## TEST 1 - do-make
function do-make () { /bin/mv -f _make.log _old_make.log; make "$@" >| _make.log 2>&1; $PAGER _make.log; }

In [33]:
## 1A) do-make
echo "Hi Mom!" > himom

linebr

do-make "himom"
## THIS SHOWS THE LATEST LOG OF do-make
cat _make.log

do-make "hidad"
cat _make.log

linebr

ls -l

--------------------------------------------------------------------------------
make: Nothing to be done for 'himom'.
make: Nothing to be done for 'himom'.
make: *** No rule to make target 'hidad'.  Stop.
make: *** No rule to make target 'hidad'.  Stop.
--------------------------------------------------------------------------------
total 16
-rw-rw-r-- 1 aveey aveey  0 Jul 20 22:07 abc
-rw-rw-r-- 1 aveey aveey  0 Jul 20 22:07 def
-rw-rw-r-- 1 aveey aveey  0 Jul 20 22:07 ghi
-rw-rw-r-- 1 aveey aveey  8 Jul 20 22:10 himom
-rw-rw-r-- 1 aveey aveey  8 Jul 20 22:06 himom.txt
-rw-rw-r-- 1 aveey aveey 49 Jul 20 22:10 _make.log
-rw-rw-r-- 1 aveey aveey 38 Jul 20 22:10 _old_make.log


In [34]:
## TEST 2 - MERGING ALIASAS & FUNCTIONS

# $ man merge
#   merge [ options ] file1 file2 file3
#   merge  incorporates all changes that lead from file2 to file3 into file1.
# NOTE: merge -p mod-file1 original mod-file2 >| new-file
alias merge='echo "do-merge MODFILE1 OLDFILE MODFILE2 > NEWFILE"'
alias do-merge='/usr/bin/merge -p'
# note: version of merge usinf diff3 to specify diff program (shell wrapper with whitespace ignored)
alias diff3-merge='/usr/bin/diff3 --merge --text --diff-program=diff.sh'
## TODO: --auto
function kdiff-merge() {
    if [ "$3" = "" ]; then
        echo "usage: $0 changed1 old changed2 output"
        return
    fi
    kdiff3 --merge --output "$4" "$1" "$2" "$3"
}

In [37]:
# 2A) merge REPLACED BY do-merge 

## INITIAL FILES TO TEST THE ALIASES
touch abc def ghi

# MESSAGE SHOWN IF MERGE IS USED
merge abc def ghi

do-merge MODFILE1 OLDFILE MODFILE2 > NEWFILE abc def ghi


In [38]:
# 2B) do-merge

## ERROR II - do-merge DOESN'T SEEM TO WORK
# $ do-merge abc def ghi
# | bash: /usr/bin/merge: No such file or directory


bash: /usr/bin/merge: No such file or directory


: 127

In [43]:
# 2C) diff3-merge

## ERROR III - diff3-merge NOT WORKING AS diff.sh MISSING
# $ diff3-merge abc def ghi
# | /usr/bin/diff3: subsidiary program 'diff.sh' not found

In [44]:
# 2D) kdiff-merge

kdiff-merge "abc" "def" "ghi" "himom.txt"
## ERROR IV - kdiff-merge NOT WORKING AS kdiff3 COMMAND MISSING
# $ kdiff-merge abc def ghi
# | bash: kdiff3: command not found

bash: kdiff3: command not found


: 127

In [50]:
# TEST 3 - WHICH FUNCTION
function which { /usr/bin/which "$@" 2> /dev/null; }
#
# full-dirname(filename): returns full path of directory for file
#
function full-dirname () { local dir=$(dirname $1); case $dir in .*) dir="$PWD/$1";; esac; echo $dir; }
#
# base-name-with-dir(file, suffix): version of basename include dir
function basename-with-dir {
    local file="$1"
    local suffix="$2"
    echo $(dirname "$file")/$(basename "$file" "$suffix");
}

In [61]:
# 3A) full-dirname RETURNS THE FULL PATH OF THE FILE
ls
linebr
full-dirname "himom.txt"

abc  def  ghi  himom  himom.txt  _make.log  _old_make.log
--------------------------------------------------------------------------------
/tmp/test-unix-alias/test-2899/himom.txt


In [62]:
# 3B) base-name-with-dir RETURNS THE BASENAME INCLUDING dir
basename-with-dir "himom.txt" 2

./himom.txt


In [63]:
## 3C) rpm-extract IS USED FOR RED HAT BASED DISTROS (RHEL, Fedora) 
function rpm-extract () { rpm2cpio "$1" | cpio --extract --make-directories; }

# # ERROR V:
# $ rpm-extract
# | bash: rpm2cpio: command not found
# | cpio: premature end of archive


bash: rpm2cpio: command not found
cpio: premature end of archive


: 2

In [64]:
# TEST 4 - DEVICE INFO ALIASES
alias dump-url='wget --recursive --relative'
#
alias gtime='/usr/bin/time'

# see 'man 5 os-release'
alias linux-version="cat /etc/os-release"
alias os-release=linux-version
alias system-status='system_status.sh -'
#

In [65]:
## 4A) dump-url GIVES 
dump-url

wget: missing URL
Usage: wget [OPTION]... [URL]...

Try `wget --help' for more options.


: 1

In [66]:
## 4B) gtime
# ERROR - VI 

# $ gtime
# | /usr/bin/time: missing program to run
# | Try '/usr/bin/time --help' for more information.



/usr/bin/time: missing program to run
Try '/usr/bin/time --help' for more information.


: 125

In [68]:
## 4C) linux-version GIVES INFO ALL INFO ABOUT THE OS INSTALLED
linux-version
linebr
# os-release ALSO WORKS SAME AS linux-version
os-release

PRETTY_NAME="Ubuntu 22.04 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=ubuntu
ID_LIKE=debian
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
UBUNTU_CODENAME=jammy
--------------------------------------------------------------------------------
PRETTY_NAME="Ubuntu 22.04 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=ubuntu
ID_LIKE=debian
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
UBUNTU_CODENAME=jammy


In [69]:
## 4D) system-status 

# ERROR VII - bad intepreter

# $ system-status
# | bash: /home/aveey/bin/system_status.sh: /bin/csh: bad interpreter: No such file or directory


bash: /home/aveey/bin/system_status.sh: /bin/csh: bad interpreter: No such file or directory


: 126